# 概要
- スクレイピングで対象ページの記事を全てdbに落とす
-  とりあえずdb (id, category, text, url)で定義する

# ライブラリimport 

In [1]:
import requests
import urllib
import sqlite3
import re
import time
from bs4 import BeautifulSoup as BS
from tqdm import tqdm_notebook as tqdm

# スクレイピング用の関数定義

In [2]:
# トップページからカテゴリ別に分けられてるページのURLを抽出
def main_url_func(main_url):
    main_url_dic = {}
    res = urllib.request.urlopen(main_url)
    data = res.read()
    text = data.decode("utf-8")
    soup = BS(text, 'html.parser')
    ul = soup.html.body.nav.ul
    li = ul.find_all('li', class_=re.compile(r'^(nav_color_).$'),)

    for i in li:    
        main_url_dic[i.a.string] = i.a.attrs['href']
    return main_url_dic


# カテゴリごとのページを訪れ、次のページへのリンクを抽出
def same_category_other_page_url(main_dic):

    for category, start_url  in main_dic.items():
        other_pages_list = [start_url]
        all_url_dic = {}
        
        url = start_url
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
    
        body = soup.html.body
        li = body.find_all("a", href=re.compile(r'^/categories/1.'))
    
        for i in li:
            other_pages_list.append("https://gunosy.com" + i.attrs['href'])
            
        all_url_dic[category] = set(other_pages_list)
        
    return all_url_dic


#　カテゴリ別のページにアクセスして記事のあるURLを抽出
def category_url_func(main_dic):
    sub_url_dic = {}
    
    for category, url in main_dic.items():
        url_list = []  
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
        soup_html = soup.html.body
        li = soup_html.find_all("a", href=re.compile(r'^https://gunosy.com/articles/'))
    
        for i in li:
            url_list.append(i.attrs['href'])
        
        sub_url_dic[category] = url_list
    
    return sub_url_dic

# 記事のtextをスクレイピング
#　URLを入力したら、textがリストで返る
def text_scraping(url):
    text_list = []
    res = urllib.request.urlopen(url)
    time.sleep(1)
    data = res.read()
    text = data.decode("utf-8")
    soup = BS(text, "html.parser")
    body = soup.html.body
    articlie = body.find_all("p")
    for i in articlie:
        text_list.append(i.string)
    return text_list

# URLの取得

In [3]:
top_url = "https://gunosy.com"
main_url_dic = main_url_func(top_url)

category_url_dic = category_url_func(main_url_dic)

In [4]:
category_url_dic.keys()

dict_keys(['エンタメ', 'スポーツ', 'おもしろ', '国内', '海外', 'コラム', 'IT・科学', 'グルメ'])

# dbの定義

In [5]:
# dbに接続する
conn = sqlite3.connect('GunosyText.db')
c = conn.cursor()

# テーブルの作成
c.execute('''CREATE TABLE textlist(id real, category real, airticle text, url text)''')

# 記事のスクレイピングと記事をdbに保存
**「逐次的にアクセスして保存」**
1. 記事のURLにアクセス
2. 記事をスクレイピング
3. 　dbに保存

In [6]:
count = 0
category_dic = {}
for name in category_url_dic.keys():
    category_dic[name] = count
    count += 1
category_dic

{'エンタメ': 0,
 'スポーツ': 1,
 'おもしろ': 2,
 '国内': 3,
 '海外': 4,
 'コラム': 5,
 'IT・科学': 6,
 'グルメ': 7}

In [ ]:
# (id, category, text, url)
index = 0

for name, url_list in category_url_dic.items():
    
    for url in tqdm(url_list):
        text = text_scraping(url)
        text = "　".join(map(str, text))
        data = [(index, category_dic[name], text, url)]
        c.executemany("INSERT INTO textlist VALUES(?, ?, ?, ?)", data)
        index += 1
    print(name + "completed" + "-"*20)
c.close()

https://gunosy.com/articles/azuHATextList_compreted
https://gunosy.com/articles/azuHATextList_compreted
https://gunosy.com/articles/RfajzTextList_compreted
https://gunosy.com/articles/RfajzTextList_compreted
https://gunosy.com/articles/auYBbTextList_compreted
https://gunosy.com/articles/auYBbTextList_compreted
https://gunosy.com/articles/RCxJ0TextList_compreted
https://gunosy.com/articles/RCxJ0TextList_compreted
https://gunosy.com/articles/aYKH7TextList_compreted
https://gunosy.com/articles/aYKH7TextList_compreted
https://gunosy.com/articles/RN5SlTextList_compreted
https://gunosy.com/articles/RN5SlTextList_compreted
https://gunosy.com/articles/RpTPMTextList_compreted
https://gunosy.com/articles/RpTPMTextList_compreted
https://gunosy.com/articles/RWK87TextList_compreted
https://gunosy.com/articles/RWK87TextList_compreted
https://gunosy.com/articles/aWO1ATextList_compreted
https://gunosy.com/articles/RvHvMTextList_compreted
https://gunosy.com/articles/RvHvMTextList_compreted
https://guno

https://gunosy.com/articles/RQYRRTextList_compreted
https://gunosy.com/articles/RQYRRTextList_compreted
https://gunosy.com/articles/RWYPlTextList_compreted
https://gunosy.com/articles/RWYPlTextList_compreted
https://gunosy.com/articles/a4WhbTextList_compreted
https://gunosy.com/articles/a4WhbTextList_compreted
https://gunosy.com/articles/apxwyTextList_compreted
https://gunosy.com/articles/apxwyTextList_compreted
https://gunosy.com/articles/RIFLMTextList_compreted
https://gunosy.com/articles/RIFLMTextList_compreted
https://gunosy.com/articles/RBjeFTextList_compreted
https://gunosy.com/articles/RBjeFTextList_compreted
https://gunosy.com/articles/abt20TextList_compreted
https://gunosy.com/articles/abt20TextList_compreted
https://gunosy.com/articles/aq7YlTextList_compreted
https://gunosy.com/articles/aq7YlTextList_compreted
https://gunosy.com/articles/aH5FRTextList_compreted
https://gunosy.com/articles/aH5FRTextList_compreted
https://gunosy.com/articles/RK8nFTextList_compreted
https://guno

https://gunosy.com/articles/aXL7hTextList_compreted
https://gunosy.com/articles/aXL7hTextList_compreted
https://gunosy.com/articles/RLO2MTextList_compreted
https://gunosy.com/articles/RLO2MTextList_compreted
https://gunosy.com/articles/RjtMhTextList_compreted
https://gunosy.com/articles/RjtMhTextList_compreted
https://gunosy.com/articles/RN7OQTextList_compreted
https://gunosy.com/articles/RN7OQTextList_compreted
https://gunosy.com/articles/aAZjzTextList_compreted
https://gunosy.com/articles/aAZjzTextList_compreted
https://gunosy.com/articles/RT7YlTextList_compreted
https://gunosy.com/articles/RT7YlTextList_compreted
https://gunosy.com/articles/R5ToQTextList_compreted
https://gunosy.com/articles/R5ToQTextList_compreted
https://gunosy.com/articles/aEXrFTextList_compreted
https://gunosy.com/articles/aEXrFTextList_compreted
https://gunosy.com/articles/a0DxBTextList_compreted
https://gunosy.com/articles/a0DxBTextList_compreted
https://gunosy.com/articles/RdaKFTextList_compreted
https://guno

https://gunosy.com/articles/amMlbTextList_compreted
https://gunosy.com/articles/amMlbTextList_compreted
https://gunosy.com/articles/R52sFTextList_compreted
https://gunosy.com/articles/R52sFTextList_compreted
https://gunosy.com/articles/Rmw3yTextList_compreted
https://gunosy.com/articles/Rmw3yTextList_compreted
https://gunosy.com/articles/a6wMhTextList_compreted
https://gunosy.com/articles/a6wMhTextList_compreted
https://gunosy.com/articles/aqKDlTextList_compreted
https://gunosy.com/articles/aqKDlTextList_compreted
https://gunosy.com/articles/acya0TextList_compreted
https://gunosy.com/articles/acya0TextList_compreted
https://gunosy.com/articles/RFv2jTextList_compreted
https://gunosy.com/articles/RFv2jTextList_compreted
https://gunosy.com/articles/RvOH7TextList_compreted
https://gunosy.com/articles/RvOH7TextList_compreted
https://gunosy.com/articles/a4w7zTextList_compreted
https://gunosy.com/articles/a4w7zTextList_compreted
https://gunosy.com/articles/RYJe4TextList_compreted
https://guno

https://gunosy.com/articles/RAYTATextList_compreted
https://gunosy.com/articles/RAYTATextList_compreted
https://gunosy.com/articles/apJN4TextList_compreted
https://gunosy.com/articles/apJN4TextList_compreted
https://gunosy.com/articles/aqBu0TextList_compreted
https://gunosy.com/articles/aqBu0TextList_compreted
https://gunosy.com/articles/afGyhTextList_compreted
https://gunosy.com/articles/afGyhTextList_compreted
https://gunosy.com/articles/R3sGMTextList_compreted
https://gunosy.com/articles/RjdQbTextList_compreted
https://gunosy.com/articles/RjdQbTextList_compreted
https://gunosy.com/articles/RYwBRTextList_compreted
https://gunosy.com/articles/RYwBRTextList_compreted
https://gunosy.com/articles/aflKQTextList_compreted
https://gunosy.com/articles/aflKQTextList_compreted
https://gunosy.com/articles/R0ZhRTextList_compreted
https://gunosy.com/articles/R0ZhRTextList_compreted
https://gunosy.com/articles/aH2JMTextList_compreted
https://gunosy.com/articles/aH2JMTextList_compreted
https://guno

https://gunosy.com/articles/RXZN4TextList_compreted
https://gunosy.com/articles/RXZN4TextList_compreted
https://gunosy.com/articles/afIIyTextList_compreted
https://gunosy.com/articles/afIIyTextList_compreted
https://gunosy.com/articles/RnFg7TextList_compreted
https://gunosy.com/articles/RnFg7TextList_compreted
https://gunosy.com/articles/RiuI7TextList_compreted
https://gunosy.com/articles/RiuI7TextList_compreted
https://gunosy.com/articles/aCyn4TextList_compreted
https://gunosy.com/articles/aCyn4TextList_compreted
https://gunosy.com/articles/RIvSMTextList_compreted
https://gunosy.com/articles/RIvSMTextList_compreted
https://gunosy.com/articles/RrHZlTextList_compreted
https://gunosy.com/articles/RrHZlTextList_compreted
https://gunosy.com/articles/RXr0RTextList_compreted
https://gunosy.com/articles/RXr0RTextList_compreted
https://gunosy.com/articles/awzbzTextList_compreted
